<a href="https://colab.research.google.com/github/seanreed1111/colab-demos/blob/master/tivdak_gpt_3_5_turbo_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!mkdir -p DATA/TIVDAK;curl https://docs.seagen.com/Tivdak_Full_Ltr_Master.pdf -o DATA/TIVDAK/tivdak.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  548k  100  548k    0     0  1781k      0 --:--:-- --:--:-- --:--:-- 1781k


In [5]:
%pip install -qq llama-index openai pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.0/580.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.8/254.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.3 MB/s eta 0:00:00


In [59]:
import llama_index
from llama_index import ListIndex, VectorStoreIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext
from llama_index.response.notebook_utils import display_response
from llama_index.llms import OpenAI
from IPython.display import Markdown, display
import logging, sys, os, json
from pathlib import Path
import openai
from typing import List, Tuple, Dict, Union

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger()
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

In [55]:
dir(llama_index)

['BeautifulSoupWebReader',
 'ChromaReader',
 'ComposableGraph',
 'DeepLakeReader',
 'DiscordReader',
 'Document',
 'DocumentSummaryIndex',
 'EmptyIndex',
 'FaissReader',
 'GPTDocumentSummaryIndex',
 'GPTEmptyIndex',
 'GPTIndexMemory',
 'GPTKeywordTableIndex',
 'GPTKnowledgeGraphIndex',
 'GPTListIndex',
 'GPTPandasIndex',
 'GPTRAKEKeywordTableIndex',
 'GPTSQLStructStoreIndex',
 'GPTSimpleKeywordTableIndex',
 'GPTTreeIndex',
 'GPTVectorStoreIndex',
 'GithubRepositoryReader',
 'GoogleDocsReader',
 'IndexStructType',
 'JSONReader',
 'KeywordExtractPrompt',
 'KeywordTableIndex',
 'KnowledgeGraphIndex',
 'LLMPredictor',
 'LangchainEmbedding',
 'ListIndex',
 'MboxReader',
 'MilvusReader',
 'MockEmbedding',
 'MockLLMPredictor',
 'NotionPageReader',
 'NullHandler',
 'ObsidianReader',
 'OpenAIEmbedding',
 'Optional',
 'PandasIndex',
 'Path',
 'PineconeReader',
 'Prompt',
 'PromptHelper',
 'PsychicReader',
 'QdrantReader',
 'QueryBundle',
 'QueryKeywordExtractPrompt',
 'QuestionAnswerPrompt',
 'R

In [16]:
test: int = 3
output: str = json.dumps({"test":test});output

'{"test": 3}'

In [7]:
cwd: Path = Path.cwd()
data_dir: Path = cwd / "DATA"
tivdak_dir: Path  = data_dir / "TIVDAK"
documents = SimpleDirectoryReader(tivdak_dir).load_data()

In [49]:
def set_open_ai_key(env_path: Union[Path, str] = None) -> Tuple[bool, str]:
  import json, os
  from pathlib import Path

  openai.api_key = None #clear previous key if exists
  env_path: Path = Path(env_path).absolute() # in case env_path is passed as a str
  if not env_path.is_file(): err: str = f"File:{str(env_path)} does not exist."
  else:
    try:
      with open(env_path, "r") as f:
          env_vars = json.load(f)
      os.environ["OPENAI_API_KEY"] = env_vars["OPENAI_API_KEY"]
      openai.api_key = os.environ["OPENAI_API_KEY"]
      openai.Model.list() #test a random command on the openai API
      err = None
    except Exception as e:
      err = json.dumps({"error":e})

  logger.info(err)
  return (True, "ok") if not err else (False, err)

In [67]:
cwd: Path = Path.cwd()
env_path: Path = cwd/ "openai.env";
print(set_open_ai_key(env_path)) # Tuple[bool, str]

(True, 'ok')


In [68]:
llm: llama_index.llms.openai.OpenAI = OpenAI(temperature=0,
             model="gpt-3.5-turbo",
             max_tokens=400,
             max_retries = 5)

service_context: llama_index.indices.service_context.ServiceContext = ServiceContext.from_defaults(llm=llm)

In [69]:
davinci_index: llama_index.indices.vector_store.base.VectorStoreIndex = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [71]:
query_engine: llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine = davinci_index.as_query_engine(response_mode="tree_summarize")
q: str = "what is tivdak used for?"
response: llama_index.response.schema.Response = query_engine.query(q)

In [72]:
display_response(response)

**`Final Response:`** TIVDAK is used to treat adults with cervical cancer that has returned or has spread to other parts of the body, and who have received chemotherapy that did not work or is no longer working.

In [76]:
q = "what is tisotumab vedotin used for?"
display_response(query_engine.query(q))

**`Final Response:`** Tisotumab vedotin is used for the treatment of adult patients with recurrent or metastatic cervical cancer with disease progression on or after chemotherapy.

In [77]:
q = "what is tisotumab used for?"
display_response(query_engine.query(q))

**`Final Response:`** Tisotumab is used for the treatment of adult patients with recurrent or metastatic cervical cancer with disease progression on or after chemotherapy.

In [79]:
q = "why do patients use cold packs when using tivdak?"
display_response(query_engine.query(q))

**`Final Response:`** Patients use cold packs when using TIVDAK to reduce the risk of eye problems.

In [80]:
q = "what are the most common adverse reactions of tivdak"
display_response(query_engine.query(q))

**`Final Response:`** The most common adverse reactions of TIVDAK are fatigue, nausea, diarrhea, abdominal pain, peripheral neuropathy, alopecia, rash, epistaxis, hemorrhage, conjunctival adverse reactions, dry eye, and corneal adverse reactions.

In [81]:
q = "what are the black box warnings of tivdak"
display_response(query_engine.query(q))

**`Final Response:`** The black box warnings of TIVDAK are not provided in the given context information.

In [82]:
q = "what are the boxed warnings of tivdak"
display_response(query_engine.query(q))

**`Final Response:`** The boxed warnings of TIVDAK are not provided in the given context information.

In [ ]:
q = "what are the most common drug interactions of tivdak"
display_response(query_engine.query(q))

In [ ]:
q = "how is tivdak administered"
display_response(query_engine.query(q))

**`Final Response:`** TIVDAK is administered as an intravenous infusion over 30 minutes every 3 weeks until disease progression or unacceptable toxicity.

In [ ]:
q = "what are the Pharmacokinetics of tivdak"
display_response(query_engine.query(q))


**`Final Response:`** The pharmacokinetics of TIVDAK are not known.

In [ ]:
q = "what are the Pharmacokinetics of Tisotumab"
display_response(query_engine.query(q))


**`Final Response:`** The pharmacokinetics of Tisotumab are not known. It is not known how tisotumab is metabolized or excreted from the body.

In [83]:
q = "what are the the exposure parameters of Tisotumab"
display_response(query_engine.query(q))


**`Final Response:`** The exposure parameters of Tisotumab Vedotin-tftv are summarized in Table 6. The mean maximum concentration (Cmax) of Tisotumab Vedotin-tftv is 40.8 μg/mL, and the mean area under the concentration-time curve (AUC) is 57.5 day*μg/mL. The mean Cmax of unconjugated MMAE (the cytotoxic component of Tisotumab Vedotin-tftv) is 5.91 ng/mL, and the mean AUC is 50 day*ng/mL.

In [ ]:
q = "what does the elimination of Tisotumab look like"
display_response(query_engine.query(q))

**`Final Response:`** The elimination of Tisotumab is characterized by a median terminal half-life of 4.04 days, a linear clearance of 1.54 L/day, and excretion of 17% of the total MMAE administered recovered in feces and 6% in urine over a 1-week period. It is thought to be metabolized in the liver and excreted in the urine and feces.

In [84]:
q = "was the efficacy of TIVDAK ever evaluated? please cite any relevant clinical studies  "
display_response(query_engine.query(q))


**`Final Response:`** Yes, the efficacy of TIVDAK was evaluated. The clinical study mentioned in the context information is "innovaTV 204." The efficacy results of TIVDAK in innovaTV 204 were evaluated using various endpoints, including Confirmed ORR (Overall Response Rate), Complete response rate, Partial response rate, and Duration of Response. The study found that the Confirmed ORR was 24%, with a 95% confidence interval of (15.9, 33.3). The Complete response rate was 7%, and the Partial response rate was 17%. The Median Duration of Response was 8.3 months, with a 95% confidence interval of (4.2, NR). These results indicate that TIVDAK showed efficacy in treating the condition being studied.